## Auto mode (single uav)

This is for testing propouses

In [1]:
# Custom modules
from config import Color
from helpers import clean  #, local2global
from helpers.change_coordinates import ENUs_to_GRAs, rel_to_abs
from mavlink.customtypes.location import ENU, ENUPose, GRAPose
from mavlink.util import save_mission
from plan import Plan
from simulators import QGC, ConfigGazebo, Gazebo, NoneVisualizer, Simulator

clean()

## Create mission

In [2]:
mission_name = "square"

enu_poses= ENUPose.list([  # east, north, up, heading
    (0., 0., 0., 0.),
    (10., 0., 0., 45.),
    (-5., -10., 0., 225.),
    (-15., 0., 0., 0.),
    (0., -20., 0., 0.),
])
colors=[
    Color.BLUE,
    Color.GREEN,
    Color.YELLOW,
    Color.ORANGE,
    Color.RED,
]
origin=GRAPose(-35.3633245, 149.1652241, 0, 0)


n_vehicles = len(enu_poses)
side_lens = (5, 5, 5, 5, 5)  # must agree with n_vehicles
rel_paths = [Plan.create_square_path(side_len=10, alt=5) for _ in enu_poses]

gra_enu_poses = ENUs_to_GRAs(origin, enu_poses)







# mission_name = "square"
# offset = ENUPose(0,0,0,0)  # east, north, up, heading 
# origin = GRAPose(-35.363261, 149.165230, 0, 0)
# rel_path = Plan.create_square_path(side_len=10, alt=5)

## Convert to Global (ENU) coordinates

In [3]:
# home = ENU(*offset[:3])
# abs_path = [rel_to_abs(pos, home, cls=ENU) for pos in rel_path]
abs_paths = [
    [rel_to_abs(pos, ENU(*offset[:3]), cls=ENU) for pos in path]
    for (path, offset) in zip(rel_paths, enu_poses)
]



## Create Plan

In [4]:
# gra_offset = ENUs_to_GRAs(origin, [offset])[0]
# gra_path = ENUs_to_GRAs(origin, abs_path)
# save_mission(name=mission_name,poses=gra_path)


gra_paths = [
    ENUs_to_GRAs(offset, abs_path)
    for offset, abs_path in zip(gra_enu_poses, abs_paths)
]

for i,gra_path in enumerate(gra_paths):
    save_mission(name=mission_name+f"_{i+1}",poses=gra_path)

## Visualization Parameters

In [5]:

markers = [
	ConfigGazebo.create_markertraj(traj=abs_path, color=color) for abs_path,color in zip(abs_paths,colors)
]
Gazebo.plot_3d_interactive(
	markers=markers,
	title="Simulation Waypoints",
	frames=(0.2, 0.2, 0.6),
	ground=-0.05
)

## Visualizer Config

In [6]:
no_vis=NoneVisualizer(origin,enu_poses)


gazebo_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway.world",
                           models=list(zip(5*["iris"],colors)),
                           marker_trajs = markers)
gaz_vis= Gazebo(origin,enu_poses,gazebo_config)

qgc_vis=QGC(origin,enu_poses)

## Launch Simulatotion

In [7]:
# TODO: Is it possible/feasible use more than one visualizer at the same time? 
# It seems there is a port conflict, only the visualizer that launches first works.

simulator = Simulator(
	visualizers=[qgc_vis],
	terminals=False,
	verbose=2
)
orac = simulator.launch(gcs_sysids={'blue 🟦': [1]})

🗺️ QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.
🚀 ArduPilot SITL vehicle 1 launched (PID 3112751)
🚀 ArduPilot SITL vehicle 3 launched (PID 3112754)
🚀 ArduPilot SITL vehicle 2 launched (PID 3112752)
🚀 ArduPilot SITL vehicle 4 launched (PID 3112756)
🚀 ArduPilot SITL vehicle 5 launched (PID 3112757)
🚀 UAV logic for vehicle 1 launched (PID 3112758)
🚀 UAV logic for vehicle 3 launched (PID 3112759)
🚀 UAV logic for vehicle 2 launched (PID 3112760)
🚀 UAV logic for vehicle 4 launched (PID 3112761)
🚀 UAV logic for vehicle 5 launched (PID 3112762)
🚀 Proxy for vehicle 1 launched (PID 3112764)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
🚀 Proxy for vehicle 3 launched (PID 3112765)
🔗 UAV logic 3 is connected to Ardupilot SITL vehicle 3
🚀 Proxy for vehicle 2 launched (PID 3112772)
🔗 UAV logic 2 is connected to Ardupilot SITL vehicle 2
🚀 Proxy for vehicle 4 launched (PID 3112774)
🔗 UAV logic 4 is connected to Ardupilot SITL vehicle 4
🚀 Proxy for vehic

bind port 5790 for SERIAL0
SERIAL0 on TCP port 5790
bind port 5800 for SERIAL0
SERIAL0 on TCP port 5800
bind port 5780 for SERIAL0
SERIAL0 on TCP port 5780
bind port 5770 for SERIAL0
SERIAL0 on TCP port 5770
bind port 5760 for SERIAL0
SERIAL0 on TCP port 5760


SIM_VEHICLE: Start
SIM_VEHICLE: Starting up at [-35.363414332037046, 149.1651690219361, 0.0, 225.0]
SIM_VEHICLE: Using defaults from (../../../ardupilot/Tools/autotest/default_params/copter.parm)
SIM_VEHICLE: Adding parameters from (/home/abeldg/repo_temp/uav-cyber-sim/params/vehicle.parm)
Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=3
Suggested EK3_DRAG_BCOEF_* = 17.209, EK3_DRAG_MCOEF = 0.209
Home: -35.363414 149.165169 alt=0.000000m hdg=225.000000
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9025
Waiting for connection ....
Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=2
Suggested EK3_DRAG_BCOEF_* = 17.209, EK3_DRAG_MCOEF = 0.209
Home: -35.363324 149.165334 alt=0.000000m hdg=45.000000
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9015
Waiting for connection ....
SIM_VEHICLE: Start
SIM_VEHICLE: Killing tasks
SIM_VEHICLE: Starting up at [-35.3633245, 149.1652241, 0.0, 0.0]
SIM_VEHICLE: Using defaults from (../../../ardupilot/Tool

bind port 5792 for SERIAL1
SERIAL1 on TCP port 5792
bind port 5793 for SERIAL2
SERIAL2 on TCP port 5793
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)
bind port 5802 for SERIAL1
SERIAL1 on TCP port 5802
bind port 5803 for SERIAL2
SERIAL2 on TCP port 5803
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)
bind port 5762 for SERIAL1
SERIAL1 on TCP port 5762
bind port 5763 for SERIAL2
SERIAL2 on TCP port 5763
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)
bind port 5782 for SERIAL1
SERIAL1 on TCP port 5782
bind port 5783 for SERIAL2
SERIAL2 on TCP port 5783
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)
bind port 5772 for SERIAL1
SERIAL1 on TCP port 5772
bind port 5773 for SERIAL2
SERIAL2 on TCP port 5773
validate_structures:528: Validating structures
Waiting for inter

New connection on SERIAL1
New connection on SERIAL1
New connection on SERIAL1
New connection on SERIAL1
New connection on SERIAL1
🔗 UAV logic 4 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 4 logic
Vehicle 4: ▶️ Plan Started: 📋 square_auto
🔗 UAV logic 5 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 5 logic
Vehicle 5: ▶️ Plan Started: 📋 square_auto
🔗 UAV logic 1 is connected to Oracle ⚪
Vehicle 4: ▶️ Action Started: 📤 UPLOAD_MISSION
✅ Heartbeat received

🚀 Starting Vehicle 1 logic
Vehicle 1: ▶️ Plan Started: 📋 square_auto
🔗 UAV logic 3 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 3 logic
Vehicle 3: ▶️ Plan Started: 📋 square_auto
Vehicle 5: ▶️ Action Started: 📤 UPLOAD_MISSION
🔗 UAV logic 2 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 2 logic
Vehicle 2: ▶️ Plan Started: 📋 square_auto
Vehicle 4: ▶️ Step Started: clear uav missions
Vehicle 1: ▶️ Action Started: 📤 UPLOAD_MISSION
🚀 GCS blue 🟦 launched (PID 31137

Vehicle 3: ▶️ Action Started: 📤 UPLOAD_MISSION


## Execute Plan

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 5: ▶️ Step Started: clear uav missions
Vehicle 2: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 1: ▶️ Step Started: clear uav missions
Vehicle 3: ▶️ Step Started: clear uav missions
Vehicle 2: ▶️ Step Started: clear uav missions
🔗 UAV logic 1 is connected
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
🧹 Cleared previous mission
Vehicle 4: ✅ Step Done: clear uav missions
🧹 Cleared previous mission
Vehicle 1: ✅ Step Done: clear uav missions
🧹 Cleared previous mission
Vehicle 5: ✅ Step Done: clear uav missions
🧹 Cleared previous mission
Vehicle 2: ✅ Step Done: clear uav missions
✅ 7 waypoints read.
🧭 Mission[0] → cmd: WAYPOINT, x: -35.3633245, y: 149.1648936, z: 0.0, current: 0
🧭 Mission[1] → cmd: TAKEOFF, x: -35.3633245, y: 149.1648936, z: 5.0, current: 0
🧭 Mission[2] → cmd: WAYPOINT, x: -35.3632347, y: 149.1648936, z: 5.0, current: 0
🧭 Miss

📤 GCS ← UAV 3: Sending DONE (attempt 67)
📤 GCS ← UAV 5: Sending DONE (attempt 41)
📤 GCS ← UAV 2: Sending DONE (attempt 5)
📤 GCS ← UAV 4: Sending DONE (attempt 2)
📤 GCS ← UAV 3: Sending DONE (attempt 68)
📤 GCS ← UAV 5: Sending DONE (attempt 42)
📤 GCS ← UAV 2: Sending DONE (attempt 6)
📤 GCS ← UAV 4: Sending DONE (attempt 3)
📤 GCS ← UAV 3: Sending DONE (attempt 69)
📤 GCS ← UAV 5: Sending DONE (attempt 43)
📤 GCS ← UAV 2: Sending DONE (attempt 7)
📤 GCS ← UAV 4: Sending DONE (attempt 4)
📤 GCS ← UAV 3: Sending DONE (attempt 70)
📤 GCS ← UAV 5: Sending DONE (attempt 44)
📤 GCS ← UAV 2: Sending DONE (attempt 8)
📤 GCS ← UAV 4: Sending DONE (attempt 5)
📤 GCS ← UAV 3: Sending DONE (attempt 71)
📤 GCS ← UAV 5: Sending DONE (attempt 45)
📤 GCS ← UAV 2: Sending DONE (attempt 9)
📤 GCS ← UAV 4: Sending DONE (attempt 6)
📤 GCS ← UAV 3: Sending DONE (attempt 72)
📤 GCS ← UAV 5: Sending DONE (attempt 46)
📤 GCS ← UAV 2: Sending DONE (attempt 10)
📤 GCS ← UAV 4: Sending DONE (attempt 7)
📤 GCS ← UAV 3: Sending DONE